# Data Transform

In this notebook, we will ask you a series of questions to evaluate your findings from your EDA. Based on your response & justification, we will ask you to also apply a subsequent data transformation. 

If you state that you will not apply any data transformations for this step, you must **justify** as to why your dataset/machine-learning does not require the mentioned data preprocessing step.

The bonus step is completely optional, but if you provide a sufficient feature engineering step in this project we will add `1000` points to your Kahoot leaderboard score.

You will write out this transformed dataframe as a `.csv` file to your `data/` folder.

**Note**: Again, note that this dataset is quite large. If you find that some data operations take too long to complete on your machine, simply use the `sample()` method to transform a subset of your data.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

In [ ]:
 # import data 
transactions = pd.read_csv("../data/bank_transactions.csv")

## Q1

Does your model contain any missing values or "non-predictive" columns? If so, which adjustments should you take to ensure that your model has good predictive capabilities? Apply your data transformations (if any) in the code-block below.

During EDA, there were no missing values in the dataset. However, some columns such as `nameOrig`, `nameDest` were identified as non-predictive, since they act as unique identifiers. These columns should be removed before training a model to ensure better predictive performance. Additionally, the `isFlaggedFraud` column was found to be unreliable, as it incorrectly flags most fraudulent transactions. This columns should be dropped as well.

In [3]:
# drop columns

transactions = transactions.drop(['nameOrig','nameDest','isFlaggedFraud'], axis=1)
transactions.head()

,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud
0,PAYMENT,983.09,36730.24,35747.15,0.00,0.00,0
1,PAYMENT,55215.25,99414.00,44198.75,0.00,0.00,0
2,CASH_IN,220986.01,7773074.97,7994060.98,924031.48,703045.48,0
3,TRANSFER,2357394.75,0.00,0.00,4202580.45,6559975.19,0
4,CASH_OUT,67990.14,0.00,0.00,625317.04,693307.19,0


## Q2

Do certain transaction types consistently differ in amount or fraud likelihood? If so, how might you transform the type column to make this pattern usable by a machine learning model? Apply your data transformations (if any) in the code-block below.

Yes, from the EDA, it was observed that TRANSFER and CASH_OUT transaction types are significantly more associated with fraudulent activity compared to others.

### ENCODING

In [4]:
# Select the column to encode
cat_feature = transactions[['type']]
cat_feature 


,type
0,PAYMENT
1,PAYMENT
2,CASH_IN
3,TRANSFER
4,CASH_OUT
...,...
999995,PAYMENT
999996,PAYMENT
999997,CASH_OUT
999998,CASH_OUT


In [5]:
# Initialize the encoder
encoder = OneHotEncoder()  

# Fit and transform
encoded_array = encoder.fit_transform(cat_feature).toarray()
encoded_array


array([[0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0.],
       ...,
       [0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.]], shape=(1000000, 5))

In [6]:
cat_names = encoder.get_feature_names_out(['type'])
cat_names



array(['type_CASH_IN', 'type_CASH_OUT', 'type_DEBIT', 'type_PAYMENT',
       'type_TRANSFER'], dtype=object)

In [7]:
# encoded dataframe
encoded_df = pd.DataFrame(encoded_array, columns=cat_names, index=transactions.index)
encoded_df.head()

,type_CASH_IN,type_CASH_OUT,type_DEBIT,type_PAYMENT,type_TRANSFER
0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,0.0
2,1.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,1.0
4,0.0,1.0,0.0,0.0,0.0


In [8]:
# only numerical columns
transactions = transactions.drop('type',axis = 1)

In [9]:
# merging numerical and encoded categorical columns
df = pd.merge(transactions, encoded_df, left_index=True, right_index=True)

df.head()

,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,type_CASH_IN,type_CASH_OUT,type_DEBIT,type_PAYMENT,type_TRANSFER
0,983.09,36730.24,35747.15,0.00,0.00,0,0.0,0.0,0.0,1.0,0.0
1,55215.25,99414.00,44198.75,0.00,0.00,0,0.0,0.0,0.0,1.0,0.0
2,220986.01,7773074.97,7994060.98,924031.48,703045.48,0,1.0,0.0,0.0,0.0,0.0
3,2357394.75,0.00,0.00,4202580.45,6559975.19,0,0.0,0.0,0.0,0.0,1.0
4,67990.14,0.00,0.00,625317.04,693307.19,0,0.0,1.0,0.0,0.0,0.0


### log transformations

To reduce skewness and make the data more suitable for modeling, a log transformation was applied to the numerical columns. This helps compress extreme values and spread out smaller ones, making the distribution more normalized.

In [10]:
# numerical columns 
skewed_cols = ['amount', 'oldbalanceOrg', 'newbalanceOrig','oldbalanceDest','newbalanceDest' ]

In [11]:
for col in skewed_cols:
    df[col + '_log10'] = np.log10(df[col].replace(0, 1))  # replace 0s with 1s to avoid log10(0)


In [12]:
df.head()

,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,type_CASH_IN,type_CASH_OUT,type_DEBIT,type_PAYMENT,type_TRANSFER,amount_log10,oldbalanceOrg_log10,newbalanceOrig_log10,oldbalanceDest_log10,newbalanceDest_log10
0,983.09,36730.24,35747.15,0.00,0.00,0,0.0,0.0,0.0,1.0,0.0,2.992593,4.565024,4.553241,0.000000,0.000000
1,55215.25,99414.00,44198.75,0.00,0.00,0,0.0,0.0,0.0,1.0,0.0,4.742059,4.997448,4.645410,0.000000,0.000000
2,220986.01,7773074.97,7994060.98,924031.48,703045.48,0,1.0,0.0,0.0,0.0,0.0,5.344365,6.890593,6.902767,5.965687,5.846983
3,2357394.75,0.00,0.00,4202580.45,6559975.19,0,0.0,0.0,0.0,0.0,1.0,6.372432,0.000000,0.000000,6.623516,6.816902
4,67990.14,0.00,0.00,625317.04,693307.19,0,0.0,1.0,0.0,0.0,0.0,4.832446,0.000000,0.000000,5.796100,5.840926


## Q3

After exploring your data, you may have noticed that fraudulent transactions are rare compared to non-fraudulent ones. What challenges might this pose when training a machine learning model? What strategies could you use to ensure your model learns meaningful patterns from the minority class? Apply your data transformations (if any) in the code-block below.

Yes, here the minority class is the fraudulent transactions,  only 1,297 out of 1,000,000 rows are labeled as fraud. This creates a class imbalance problem, where the model may become biased toward predicting the majority class (non-fraud), potentially missing actual fraud cases.

To address class imbalance, we can apply techniques like SMOTE

## Bonus (optional)

Are there interaction effects between variables (e.g., fraud and high amount and transaction type) that aren't captured directly in the dataset? Would it be helpful to manually engineer any new features that reflect these interactions? Apply your data transformations (if any) in the code-block below.

Yes, during EDA, we noticed that fraudulent transactions often involved high amounts and occurred mostly in TRANSFER or CASH_OUT types. This suggests an interaction effect between transaction type and amount, which isn’t explicitly captured in the dataset but could be valuable for modeling.
This means a combination of certain features (like high amount and type) might be a better indicator of fraud than either one alone.

In [13]:
# Create new interaction column
df['amount_transfer'] = df['amount'] * df['type_TRANSFER']

# apply log10
df['amount_transfer_log10'] = np.log10(df['amount_transfer'].replace(0, 1)) 



In [14]:
df.head()

,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,type_CASH_IN,type_CASH_OUT,type_DEBIT,type_PAYMENT,type_TRANSFER,amount_log10,oldbalanceOrg_log10,newbalanceOrig_log10,oldbalanceDest_log10,newbalanceDest_log10,amount_transfer,amount_transfer_log10
0,983.09,36730.24,35747.15,0.00,0.00,0,0.0,0.0,0.0,1.0,0.0,2.992593,4.565024,4.553241,0.000000,0.000000,0.00,0.000000
1,55215.25,99414.00,44198.75,0.00,0.00,0,0.0,0.0,0.0,1.0,0.0,4.742059,4.997448,4.645410,0.000000,0.000000,0.00,0.000000
2,220986.01,7773074.97,7994060.98,924031.48,703045.48,0,1.0,0.0,0.0,0.0,0.0,5.344365,6.890593,6.902767,5.965687,5.846983,0.00,0.000000
3,2357394.75,0.00,0.00,4202580.45,6559975.19,0,0.0,0.0,0.0,0.0,1.0,6.372432,0.000000,0.000000,6.623516,6.816902,2357394.75,6.372432
4,67990.14,0.00,0.00,625317.04,693307.19,0,0.0,1.0,0.0,0.0,0.0,4.832446,0.000000,0.000000,5.796100,5.840926,0.00,0.000000


In [15]:
# save to .csv file

df.to_csv('../data/transformed_data.csv',index=False)